In [1]:
import pandas as pd
import numpy as np
import glob
import re

from datetime import date
from astropy.io import fits

The goal of this notebook is to have data ready for the exploratory data analysis.  
We will extract the required data from our data sources and store them in a SQLite database.

Our data sources are:

1. The cumulative KOIs Activity Table, a CSV file downloaded from the Exoplanet Archive website:  
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative

2. The Data Validation FITS files, available on the Mikulski Archive Space Telescopes (MAST) website:    
https://archive.stsci.edu/missions/kepler/dv_files/tar_files_dr25/

The role of the cumulative KOIs acitivity table is to provide the list of KOIs, the classification target variable and the features justifying the classification.  
The data validation FITS files provide all the features: the stellar properties and transit parameters.

Notes:

* The features are extracted from more than 140Go of FITS files, not provided in this project for evident reason
* This notebook is a technical notebook and doesn't explain the data sources, notably the FITS files explained in detail in the `Annex_Data_Validation_FITS_Files.ipynb` notebook

# 1. The Cumulative Kepler Objects of Interest Activity Table

## 1.1 Load the data source

In [2]:
# Load the source Cumulative KOIs Activity Table (skip the first 75 rows containing the column descriptions)
filename = '../resources/source_data/cumulative_kois_activity_table.csv'
kois = pd.read_csv(filename, skiprows=36)

# Format the kepler ids (length 9, padded with 0)
kois['kepid'] = kois['kepid'].astype(str)
kois['kepid'] = kois['kepid'].str.zfill(9)

# Add a column to store the FITS filename
kois['filename']  = ""

# Show data
print('KOIs Activity Table Shape:', kois.shape)
kois.head(5)

KOIs Activity Table Shape: (8054, 29)


,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,...,koi_tce_plnt_num,koi_tce_delivname,koi_quarters,koi_bin_oedp_sig,koi_trans_mod,koi_model_dof,koi_model_chisq,koi_datalink_dvr,koi_datalink_dvs,filename
0,010797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,...,1,q1_q17_dr25_tce,11111111111111111000000000000000,0.6864,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-001-2016...,
1,010797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,0,...,2,q1_q17_dr25_tce,11111111111111111000000000000000,0.0023,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-002-2016...,
2,010811496,K00753.01,NaN,CANDIDATE,Done,2018-08-16,CANDIDATE,0.000,0,0,...,1,q1_q17_dr25_tce,11111101110111011000000000000000,0.6624,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010811/010811496/dv/kplr010811496-20160209...,010/010811/010811496/dv/kplr010811496-001-2016...,
3,010848459,K00754.01,NaN,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.000,0,1,...,1,q1_q17_dr25_tce,11111110111011101000000000000000,0.0000,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010848/010848459/dv/kplr010848459-20160209...,010/010848/010848459/dv/kplr010848459-001-2016...,
4,010854555,K00755.01,Kepler-664 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,...,1,q1_q17_dr25_tce,01111111111111111000000000000000,0.3090,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010854/010854555/dv/kplr010854555-20160209...,010/010854/010854555/dv/kplr010854555-001-2016...,


## 1.2  Add the FITS filename 

Each observation in the KOIs Activity Table is a registred TCE identified by its full KOI Name (`kepoi_name` column) around a star identified by the first part of the KOI Name but also its KIC number (`kepid` column). All the TCE's features are stored in FITS files containing the KIC number in their filenames.

In this section, we map the corresponding FITS filename to observations for which we have FITS file:

In [3]:
# Path at which the FITS files are stored
FITS_PATH = '/Users/grillety/Exoplanets/Fits'

In [4]:
# Get the list of files with a given extension from a base path
def get_list_of_path(base_path, extension, recursive=True):
    return glob.glob(base_path + '/**/*.' + extension, recursive=recursive)

# Get the list of all .FITS files stored on my disk drive
fits_files = get_list_of_path(FITS_PATH, 'fits')
fits_files[:5]

['/Users/grillety/Exoplanets/Fits/kplr005513897-20160128150956_dvt.fits',
 '/Users/grillety/Exoplanets/Fits/kplr007898352-20160128150956_dvt.fits',
 '/Users/grillety/Exoplanets/Fits/kplr012416597-20160128150956_dvt.fits',
 '/Users/grillety/Exoplanets/Fits/kplr010907307-20160128150956_dvt.fits',
 '/Users/grillety/Exoplanets/Fits/kplr010864656-20160128150956_dvt.fits']

In [5]:
# Add the corresponding FITS filename to the KOI observations
for file in fits_files:
    
    # Extract the filename
    filename = file.split('/')[-1]
    
    # Extract the KIC number from the filename
    kic = re.search(r"(?<=kplr).*(?=-)", filename).group(0)
    
    # Update the dataframe with the filename corresponding to the kepler id
    kois.loc[kois['kepid'] == kic, 'filename'] = filename

## 1.3 Basic Cleaning

### 1.3.1 Drop observations with missing FITS files and integrity issues

We must drop TCEs without FITS files and integrity issues (no TCE registred, 0 transit, etc.).  
The rules allowing this cleaning have been discovered iteratively during the first attempts to build the datasets.

In [6]:
# Drop observations for which we have no fits file
filter = (kois['filename'] == "") 
kois.drop(index=kois[filter].index, inplace=True)
print(kois.shape)

(8054, 29)


In [7]:
# Drop observations with missing values in the TCE Number and Score columns
kois.dropna(subset=['koi_tce_plnt_num', 'koi_score'], axis=0, inplace=True)
print(kois.shape)

(8054, 29)


In [8]:
# Drop observations containing NOFITS in their comment
filter = kois['koi_comment'].str.contains('NOFITS')
kois.drop(index=kois[filter].index, inplace=True)
print(kois.shape)

(7745, 29)


In [9]:
# Drop observation with 0 transits
filter = (kois['koi_num_transits'] == 0)
kois.drop(index=kois[filter].index, inplace=True)
print(kois.shape)

(7737, 29)


In [10]:
# Drop strange observations for which we have FITS file but no transit model inside
# No column has been found to exclude them automatically
missing_model_kepois = ['K03158.01', 'K03156.02', 'K03175.01', 'K07344.01', 'K06460.01']
filter = kois['kepoi_name'].isin(missing_model_kepois)
kois.drop(index=kois[filter].index, inplace=True)
print(kois.shape)

(7732, 29)


In [11]:
# Reset the dataframe index
kois.reset_index(inplace=True, drop=True)

### 1.3.2 Drop the empty variables

In [12]:
# Get the empty variables
vars_novalue = list(kois.columns[kois.isnull().all()])

# Drop the empty variables
kois.dropna(axis=1, how='all', inplace=True)

print('Empty variables dropped:', vars_novalue)
print('\nDataFrame Shape:', kois.shape)

Empty variables dropped: ['koi_model_dof', 'koi_model_chisq']

DataFrame Shape: (7732, 27)


### 1.3.3 Drop the variables with only one unique value

#### Object variables

In [13]:
# Get the object variables containing only one unique value
s_unique = kois.describe(include=np.object).T.loc[:, 'unique']
vars_unique = s_unique[s_unique == 1]

# Drop object variables with one unique values
kois.drop(axis=1, columns=vars_unique.index, inplace=True)

print('Object variables with only one unique value dropped:', list(vars_unique.index))
print('\nDataFrame Shape:', kois.shape)

Object variables with only one unique value dropped: ['koi_vet_stat', 'koi_vet_date', 'koi_disp_prov', 'koi_tce_delivname', 'koi_trans_mod']

DataFrame Shape: (7732, 22)


#### Numeric variables

In [14]:
# Get statistics about the number variables
df_stats = kois.describe(include=np.number).T

# Variable with only on unique value (min value = max value)
vars_unique = df_stats.loc[(df_stats['min'] == df_stats['max']), :].index

# Drop object variables with only one unique values
kois.drop(axis=1, columns=vars_unique, inplace=True)

print('Numerical variables with only one unique value dropped:', list(vars_unique))
print('\nDataFrame Shape:', kois.shape)

Numerical variables with only one unique value dropped: []

DataFrame Shape: (7732, 22)


## 1.4 Save the target and justification variables in a database table

We are interested to keep only the classification target variable and the features justifiying it, only available in the KOIs activity table:
* the classification target variable (`koi_disposition`)
* the classifcation using kepler data variable (`koi_pdisposition`)
* the score
* the false positive flags
* the comment

The other features was necessary to link the target variable with the FITS files from which the features will be extracted.  
They will reappear during the data extraction from the TCE extension headers.

In [15]:
import sqlite3

# Connect to the database
DB_PATH = '../resources/capstone_db.sqlite'
db = sqlite3.connect(DB_PATH)

# Get a cursor
cursor = db.cursor()

In [16]:
# 1. CREATE A DATAFRAME WITH ONLY THE TARGET AND JUSTIFICATION VARIABLES
# ----------------------------------------------------------------------

# Target and justification variables coming from the KOIs activity table
cols = [
    'kepoi_name',
    'kepler_name',
    'koi_disposition',
    'koi_pdisposition',
    'koi_score',
    'koi_fpflag_nt',
    'koi_fpflag_ss',
    'koi_fpflag_co',
    'koi_fpflag_ec',
    'koi_comment',
    'filename'
]

# Create a dataframe with only the stellar properties variables 
df = kois.reindex(columns=cols)



# 2. CREATE THE DATABASE TABLE
# ----------------------------

# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS kois_activity_table;')

# Create the kois_activity_table table
query = '''
CREATE TABLE kois_activity_table
(
    kepoi_name TEXT PRIMARY KEY,
    kepler_name TEXT,
    koi_disposition TEXT NOT NULL,
    koi_pdisposition TEXT NOT NULL,
    koi_score REAL NOT NULL,
    koi_fpflag_nt INTEGER NOT NULL,
    koi_fpflag_ss INTEGER NOT NULL,
    koi_fpflag_co  INTEGER NOT NULL,
    koi_fpflag_ec INTEGER NOT NULL,
    koi_comment TEXT,
    filename TEXT
);
'''
cursor.execute(query)


# 3. LOAD THE DATA IN THE TABLE
# -----------------------------

# Insert the source data in the fits_stellar_properties database table
df.to_sql(name='kois_activity_table', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    kois_activity_table;
'''
df = pd.read_sql_query(query, db)
print('Database Table Shape: ', df.shape)
df.head()

Database Table Shape:  (7732, 11)


,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_comment,filename
0,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,NO_COMMENT,kplr010797460-20160128150956_dvt.fits
1,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,NO_COMMENT,kplr010797460-20160128150956_dvt.fits
2,K00753.01,None,CANDIDATE,CANDIDATE,0.000,0,0,0,0,DEEP_V_SHAPED,kplr010811496-20160128150956_dvt.fits
3,K00754.01,None,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,MOD_ODDEVEN_DV---MOD_ODDEVEN_ALT---DEEP_V_SHAPED,kplr010848459-20160128150956_dvt.fits
4,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,NO_COMMENT,kplr010854555-20160128150956_dvt.fits


# 2. The Data Validation FITS Files

The `Annex_Data_Validation_FITS_Files.ipynb` notebook explains in detail the FITS file format.

We have to:

1. Extract the stellar properties from the FITS files primary header
2. Extract the transit parameters from the FITS files TCE extension headers

There are one FITS file per hosting star.  
A star can be the host of many TCEs, so a FITS file can have many TCE extension headers, one for each registred TCE.

## 2.1 Get the list of keys to extract from the primary and TCE extension headers

In [17]:
# FITS file used to get the list of features/keys to extract from the primary and tce extension headers
filepath = '../resources/source_data/kplr010797460-20160128150956_dvt.fits'

# Get the primary header keys/features to extract
header_keys = []
with fits.open(filepath, mode='readonly') as hdulist:
    # Get all the keys stored in the primary header
    header_keys = list(hdulist[0].header.keys())
    
    # Keep only keys relative to the KOI star and drop keys describing the FITS file structure
    header_keys = header_keys[7:-1]
    
# Get the extension header keys/features to extract
extension_keys = []
with fits.open(filepath, mode='readonly') as hdulist:
    
    # Get all the keys stored in the extension header
    extension_keys = list(hdulist[1].header.keys())
    
    # Keep only keys relative to the TCE and drop keys describing the FITS file structure and duplicated key from the primary header
    extension_keys = extension_keys[58:-1]

## 2.2 Extract the data from the primary and TCE extension headers

In [18]:
# Exctrat the data from the FITS files and store them in a temporary list
data = []
for i in range(kois.shape[0]):
    
    # Get filename and TCE number
    filename = kois.filename[i]
    tce_number = int(kois.koi_tce_plnt_num[i])
    
    # Print progress status
    print('{}: {} - {}'.format(i, filename, tce_number))
    
    # FIT filepath
    filepath = FITS_PATH + '/' + filename
    
    with fits.open(filepath, mode='readonly') as hdulist:
        
        fits_values = [kois.kepoi_name[i], kois.koi_disposition[i], kois.kepid[i], kois.filename[i]]
        
        # Get primary header features including the stellar properties
        for key in header_keys:
            fits_values.append(hdulist[0].header[key])
        
        # Get extension header features including the transit parameters
        for key in extension_keys:
            fits_values.append(hdulist[tce_number].header[key])
            
        data.append(fits_values)

0: kplr010797460-20160128150956_dvt.fits - 1
1: kplr010797460-20160128150956_dvt.fits - 2
2: kplr010811496-20160128150956_dvt.fits - 1
3: kplr010848459-20160128150956_dvt.fits - 1
4: kplr010854555-20160128150956_dvt.fits - 1
5: kplr010872983-20160128150956_dvt.fits - 1
6: kplr010872983-20160128150956_dvt.fits - 2
7: kplr010872983-20160128150956_dvt.fits - 3
8: kplr006721123-20160128150956_dvt.fits - 1
9: kplr010910878-20160128150956_dvt.fits - 1
10: kplr011446443-20160128150956_dvt.fits - 1
11: kplr010666592-20160128150956_dvt.fits - 1
12: kplr006922244-20160128150956_dvt.fits - 1
13: kplr010984090-20160128150956_dvt.fits - 2
14: kplr010419211-20160128150956_dvt.fits - 1
15: kplr010464078-20160128150956_dvt.fits - 1
16: kplr010480982-20160128150956_dvt.fits - 1
17: kplr010485250-20160128150956_dvt.fits - 1
18: kplr010526549-20160128150956_dvt.fits - 1
19: kplr010583066-20160128150956_dvt.fits - 1
20: kplr010583180-20160128150956_dvt.fits - 1
21: kplr010601284-20160128150956_dvt.fits - 

178: kplr006849310-20160128150956_dvt.fits - 2
179: kplr006849310-20160128150956_dvt.fits - 3
180: kplr006862328-20160128150956_dvt.fits - 1
181: kplr009471974-20160128150956_dvt.fits - 2
182: kplr006862603-20160128150956_dvt.fits - 1
183: kplr007366258-20160128150956_dvt.fits - 2
184: kplr007366258-20160128150956_dvt.fits - 1
185: kplr007366258-20160128150956_dvt.fits - 3
186: kplr007366258-20160128150956_dvt.fits - 4
187: kplr007373451-20160128150956_dvt.fits - 1
188: kplr006948054-20160128150956_dvt.fits - 3
189: kplr006949607-20160128150956_dvt.fits - 1
190: kplr006949607-20160128150956_dvt.fits - 2
191: kplr007031517-20160128150956_dvt.fits - 1
192: kplr011869052-20160128150956_dvt.fits - 1
193: kplr007109675-20160128150956_dvt.fits - 1
194: kplr007109675-20160128150956_dvt.fits - 2
195: kplr007118364-20160128150956_dvt.fits - 1
196: kplr007134976-20160128150956_dvt.fits - 1
197: kplr007134976-20160128150956_dvt.fits - 2
198: kplr007135852-20160128150956_dvt.fits - 1
199: kplr0072

365: kplr005956342-20160128150956_dvt.fits - 1
366: kplr005956342-20160128150956_dvt.fits - 2
367: kplr005956342-20160128150956_dvt.fits - 3
368: kplr005956342-20160128150956_dvt.fits - 4
369: kplr005956656-20160128150956_dvt.fits - 1
370: kplr003109930-20160128150956_dvt.fits - 1
371: kplr002854914-20160128150956_dvt.fits - 1
372: kplr002854914-20160128150956_dvt.fits - 2
373: kplr003337425-20160128150956_dvt.fits - 1
374: kplr003116412-20160128150956_dvt.fits - 1
375: kplr003865567-20160128150956_dvt.fits - 1
376: kplr003544689-20160128150956_dvt.fits - 1
377: kplr004037164-20160128150956_dvt.fits - 1
378: kplr003443790-20160128150956_dvt.fits - 1
379: kplr003966912-20160128150956_dvt.fits - 1
380: kplr008394721-20160128150956_dvt.fits - 1
381: kplr001161345-20160128150956_dvt.fits - 1
382: kplr010227501-20160128150956_dvt.fits - 1
383: kplr002854698-20160128150956_dvt.fits - 1
384: kplr002854698-20160128150956_dvt.fits - 2
385: kplr008559644-20160128150956_dvt.fits - 2
386: kplr0058

554: kplr010514770-20160128150956_dvt.fits - 1
555: kplr009775938-20160128150956_dvt.fits - 2
556: kplr009787239-20160128150956_dvt.fits - 1
557: kplr009787239-20160128150956_dvt.fits - 3
558: kplr009787239-20160128150956_dvt.fits - 2
559: kplr009787239-20160128150956_dvt.fits - 4
560: kplr009787239-20160128150956_dvt.fits - 5
561: kplr002309719-20160128150956_dvt.fits - 1
562: kplr002558363-20160128150956_dvt.fits - 1
563: kplr002716853-20160128150956_dvt.fits - 1
564: kplr002445154-20160128150956_dvt.fits - 1
565: kplr002715135-20160128150956_dvt.fits - 1
566: kplr008644288-20160128150956_dvt.fits - 3
567: kplr003113266-20160128150956_dvt.fits - 1
568: kplr003247268-20160128150956_dvt.fits - 1
569: kplr003247268-20160128150956_dvt.fits - 2
570: kplr003232859-20160128150956_dvt.fits - 1
571: kplr003098184-20160128150956_dvt.fits - 1
572: kplr010342065-20160128150956_dvt.fits - 1
573: kplr010352945-20160128150956_dvt.fits - 1
574: kplr010354039-20160128150956_dvt.fits - 1
575: kplr0103

734: kplr006774826-20160128150956_dvt.fits - 1
735: kplr005652237-20160128150956_dvt.fits - 1
736: kplr007211469-20160128150956_dvt.fits - 1
737: kplr007375795-20160128150956_dvt.fits - 1
738: kplr007255336-20160128150956_dvt.fits - 1
739: kplr007831264-20160128150956_dvt.fits - 1
740: kplr007802136-20160128150956_dvt.fits - 1
741: kplr007532973-20160128150956_dvt.fits - 1
742: kplr009632895-20160128150956_dvt.fits - 1
743: kplr007449844-20160128150956_dvt.fits - 1
744: kplr008766222-20160128150956_dvt.fits - 1
745: kplr008678594-20160128150956_dvt.fits - 1
746: kplr010925104-20160128150956_dvt.fits - 3
747: kplr008678594-20160128150956_dvt.fits - 2
748: kplr008703884-20160128150956_dvt.fits - 1
749: kplr008560840-20160128150956_dvt.fits - 1
750: kplr004076098-20160128150956_dvt.fits - 1
751: kplr004551328-20160128150956_dvt.fits - 1
752: kplr004282872-20160128150956_dvt.fits - 1
753: kplr004639868-20160128150956_dvt.fits - 1
754: kplr010555375-20160128150956_dvt.fits - 1
755: kplr0043

...

6854: kplr006386784-20160128150956_dvt.fits - 1
6855: kplr006390236-20160128150956_dvt.fits - 1
6856: kplr006421188-20160128150956_dvt.fits - 1
6857: kplr007116236-20160128150956_dvt.fits - 1
6858: kplr007117513-20160128150956_dvt.fits - 1
6859: kplr007117541-20160128150956_dvt.fits - 1
6860: kplr007118545-20160128150956_dvt.fits - 1
6861: kplr007125636-20160128150956_dvt.fits - 1
6862: kplr007128918-20160128150956_dvt.fits - 1
6863: kplr007132015-20160128150956_dvt.fits - 1
6864: kplr008088354-20160128150956_dvt.fits - 1
6865: kplr008091197-20160128150956_dvt.fits - 1
6866: kplr008094140-20160128150956_dvt.fits - 1
6867: kplr008094221-20160128150956_dvt.fits - 1
6868: kplr008097825-20160128150956_dvt.fits - 1
6869: kplr008098515-20160128150956_dvt.fits - 1
6870: kplr008098960-20160128150956_dvt.fits - 1
6871: kplr008104436-20160128150956_dvt.fits - 1
6872: kplr008822585-20160128150956_dvt.fits - 1
6873: kplr008823397-20160128150956_dvt.fits - 1
6874: kplr008845221-20160128150956_dvt.f

7029: kplr005978559-20160128150956_dvt.fits - 2
7030: kplr008838950-20160128150956_dvt.fits - 2
7031: kplr006867555-20160128150956_dvt.fits - 2
7032: kplr012208631-20160128150956_dvt.fits - 2
7033: kplr003346543-20160128150956_dvt.fits - 1
7034: kplr003352751-20160128150956_dvt.fits - 1
7035: kplr003356193-20160128150956_dvt.fits - 1
7036: kplr003427776-20160128150956_dvt.fits - 1
7037: kplr003437739-20160128150956_dvt.fits - 1
7038: kplr003441340-20160128150956_dvt.fits - 1
7039: kplr003448323-20160128150956_dvt.fits - 1
7040: kplr004863369-20160128150956_dvt.fits - 1
7041: kplr004863753-20160128150956_dvt.fits - 1
7042: kplr004908495-20160128150956_dvt.fits - 1
7043: kplr004912589-20160128150956_dvt.fits - 1
7044: kplr004914709-20160128150956_dvt.fits - 1
7045: kplr004914737-20160128150956_dvt.fits - 1
7046: kplr005471415-20160128150956_dvt.fits - 1
7047: kplr005471480-20160128150956_dvt.fits - 1
7048: kplr005479973-20160128150956_dvt.fits - 1
7049: kplr005513833-20160128150956_dvt.f

7204: kplr008283875-20160128150956_dvt.fits - 1
7205: kplr009119405-20160128150956_dvt.fits - 1
7206: kplr009163674-20160128150956_dvt.fits - 1
7207: kplr009172506-20160128150956_dvt.fits - 1
7208: kplr009179531-20160128150956_dvt.fits - 1
7209: kplr009965691-20160128150956_dvt.fits - 1
7210: kplr009971475-20160128150956_dvt.fits - 1
7211: kplr010000490-20160128150956_dvt.fits - 1
7212: kplr010002049-20160128150956_dvt.fits - 1
7213: kplr010014702-20160128150956_dvt.fits - 1
7214: kplr010019854-20160128150956_dvt.fits - 1
7215: kplr010020423-20160128150956_dvt.fits - 1
7216: kplr011027722-20160128150956_dvt.fits - 1
7217: kplr011029877-20160128150956_dvt.fits - 1
7218: kplr011068630-20160128150956_dvt.fits - 1
7219: kplr011076279-20160128150956_dvt.fits - 1
7220: kplr011092783-20160128150956_dvt.fits - 1
7221: kplr011124509-20160128150956_dvt.fits - 1
7222: kplr012506342-20160128150956_dvt.fits - 1
7223: kplr012555616-20160128150956_dvt.fits - 1
7224: kplr012557713-20160128150956_dvt.f

7394: kplr010461045-20160128150956_dvt.fits - 1
7395: kplr010493253-20160128150956_dvt.fits - 1
7396: kplr010515564-20160128150956_dvt.fits - 1
7397: kplr010321927-20160128150956_dvt.fits - 1
7398: kplr010321972-20160128150956_dvt.fits - 1
7399: kplr010322797-20160128150956_dvt.fits - 1
7400: kplr010331279-20160128150956_dvt.fits - 1
7401: kplr010338529-20160128150956_dvt.fits - 1
7402: kplr010340212-20160128150956_dvt.fits - 1
7403: kplr010383729-20160128150956_dvt.fits - 1
7404: kplr005709226-20160128150956_dvt.fits - 1
7405: kplr005775090-20160128150956_dvt.fits - 1
7406: kplr005791705-20160128150956_dvt.fits - 1
7407: kplr005802205-20160128150956_dvt.fits - 1
7408: kplr005812648-20160128150956_dvt.fits - 6
7409: kplr005858919-20160128150956_dvt.fits - 1
7410: kplr005872139-20160128150956_dvt.fits - 1
7411: kplr010519701-20160128150956_dvt.fits - 1
7412: kplr010664150-20160128150956_dvt.fits - 1
7413: kplr010735575-20160128150956_dvt.fits - 1
7414: kplr010753394-20160128150956_dvt.f

7586: kplr011612241-20160128150956_dvt.fits - 3
7587: kplr011661803-20160128150956_dvt.fits - 1
7588: kplr011661803-20160128150956_dvt.fits - 2
7589: kplr008608544-20160128150956_dvt.fits - 1
7590: kplr008608544-20160128150956_dvt.fits - 2
7591: kplr008618147-20160128150956_dvt.fits - 1
7592: kplr008630840-20160128150956_dvt.fits - 1
7593: kplr008683144-20160128150956_dvt.fits - 1
7594: kplr008708961-20160128150956_dvt.fits - 1
7595: kplr008740744-20160128150956_dvt.fits - 1
7596: kplr007451315-20160128150956_dvt.fits - 1
7597: kplr007751294-20160128150956_dvt.fits - 1
7598: kplr007832787-20160128150956_dvt.fits - 1
7599: kplr008008913-20160128150956_dvt.fits - 1
7600: kplr008022520-20160128150956_dvt.fits - 1
7601: kplr008108450-20160128150956_dvt.fits - 2
7602: kplr008293631-20160128150956_dvt.fits - 1
7603: kplr009604563-20160128150956_dvt.fits - 1
7604: kplr009639021-20160128150956_dvt.fits - 1
7605: kplr009640649-20160128150956_dvt.fits - 1
7606: kplr009643210-20160128150956_dvt.f

In [19]:
# Create a dataframe from the list of records
df_fits = pd.DataFrame.from_records(data)

# Rename the columns in lower case and reorder them
header_keys = [key.lower() for key in header_keys]
extension_keys = [key.lower() for key in extension_keys]
df_fits.columns = ['kepoi_name', 'koi_disposition', 'kepid', 'filename'] + header_keys + extension_keys

print('Dataframe shape:', df_fits.shape)
df_fits.head()

Dataframe shape: (7732, 99)


,kepoi_name,koi_disposition,kepid,filename,origin,date,creator,procver,filever,timversn,...,impact,inclin,drratio,radratio,pradius,maxmes,maxses,ntrans,convrge,meddetr
0,K00752.01,CONFIRMED,010797460,kplr010797460-20160128150956_dvt.fits,NASA/Ames,2016-03-01,1165179 DvTimeSeriesExporter2PipelineModule,svn+ssh://murzim/repo/soc/tags/release/9.3.43 ...,2.0,OGIP/93-003,...,0.934177,84.169681,9.196221,0.028956,2.929099,28.470819,5.135849,142,True,17.639999
1,K00752.02,CONFIRMED,010797460,kplr010797460-20160128150956_dvt.fits,NASA/Ames,2016-03-01,1165179 DvTimeSeriesExporter2PipelineModule,svn+ssh://murzim/repo/soc/tags/release/9.3.43 ...,2.0,OGIP/93-003,...,0.711630,89.395301,67.428761,0.029346,2.968514,20.109507,7.027669,25,True,23.520000
2,K00753.01,CANDIDATE,010811496,kplr010811496-20160128150956_dvt.fits,NASA/Ames,2016-03-01,1165192 DvTimeSeriesExporter2PipelineModule,svn+ssh://murzim/repo/soc/tags/release/9.3.43 ...,2.0,OGIP/93-003,...,0.929919,89.029114,54.880784,0.129501,12.266182,187.449097,37.159767,56,True,9.800000
3,K00754.01,FALSE POSITIVE,010848459,kplr010848459-20160128150956_dvt.fits,NASA/Ames,2016-03-01,1165166 DvTimeSeriesExporter2PipelineModule,svn+ssh://murzim/repo/soc/tags/release/9.3.43 ...,2.0,OGIP/93-003,...,0.933470,74.673384,3.531577,0.108508,9.365962,541.895081,39.066551,621,True,11.760000
4,K00755.01,CONFIRMED,010854555,kplr010854555-20160128150956_dvt.fits,NASA/Ames,2016-03-01,1165185 DvTimeSeriesExporter2PipelineModule,svn+ssh://murzim/repo/soc/tags/release/9.3.43 ...,2.0,OGIP/93-003,...,0.690533,85.604302,9.009601,0.023557,2.688826,33.191898,4.749945,515,True,8.820000


The dataframe contains features other than the stellar properties and transit parameters that we will use to train models.  
These supplementary features could be usefull during the EDA and data cleaning processes.

## 2.3 Basic Cleaning

### 2.3.1 Drop the empty variables

In [20]:
# Get the variables with no value
vars_novalue = list(df_fits.columns[df_fits.isnull().all()])

# Drop variables with no value
df_fits.dropna(axis=1, how='all', inplace=True)

print('Empty variables dropped:', vars_novalue)
print('\nDataFrame Shape:', df_fits.shape)

Empty variables dropped: ['tierabso']

DataFrame Shape: (7732, 98)


### 2.3.2 Drop the variables with only one unique value

#### Object variables

In [21]:
# Get the object variables containing only one unique value
s_unique = df_fits.describe(include=np.object).T.loc[:, 'unique']
vars_unique = s_unique[s_unique == 1]

# Drop object variables with one unique values
df_fits.drop(axis=1, columns=vars_unique.index, inplace=True)

print('Object variables with only one unique value dropped:', list(vars_unique.index))
print('\nDataFrame Shape:', df_fits.shape)

Object variables with only one unique value dropped: ['origin', 'date', 'procver', 'filever', 'timversn', 'telescop', 'instrume', 'obsmode', 'mission', 'radesys', 'xmlstr', 'dvversn', 'timeref', 'tassign', 'timesys', 'timeunit', 'date-obs', 'date-end']

DataFrame Shape: (7732, 80)


#### Numeric variables

In [22]:
# Get statistics about the number variables
df_stats = df_fits.describe(include=np.number).T

# Variable with only on unique value (min value = max value)
vars_unique = df_stats.loc[(df_stats['min'] == df_stats['max']), :].index

# Drop object variables with only one unique values
df_fits.drop(axis=1, columns=vars_unique, inplace=True)

print('Numerical variables with only one unique value dropped:', list(vars_unique))
print('\nDataFrame Shape:', df_fits.shape)

Numerical variables with only one unique value dropped: ['data_rel', 'equinox', 'bjdrefi', 'bjdreff', 'lc_start', 'lc_end', 'deadc', 'timepixr', 'tierrela', 'int_time', 'readtime', 'frametim', 'num_frm', 'timedel', 'nreadout']

DataFrame Shape: (7732, 65)


#### Boolean variables

In [23]:
# Get the boolean variables containing only one unique value
s_unique = df_fits.describe(include=np.bool).T.loc[:, 'unique']
vars_unique = s_unique[s_unique == 1]

# Drop boolean variables with one unique values
df_fits.drop(axis=1, columns=vars_unique.index, inplace=True)

print('Boolean variables with only one unique value dropped:', list(vars_unique.index))
print('\nDataFrame Shape:', df_fits.shape)

Boolean variables with only one unique value dropped: ['backapp', 'deadapp', 'vignapp']

DataFrame Shape: (7732, 62)


### 2.3.3 Variables with more than 90% of missing values

In [24]:
threshold = 0.9 * df_fits.shape[0]
vars = df_fits.columns[df_fits.isnull().sum() > threshold]
df_fits.dropna(axis=1, thresh=threshold, inplace=True)

print('Variables dropped:', list(vars))
print('\nDataFrame Shape:', df_fits.shape)

Variables dropped: ['parallax', 'scpid']

DataFrame Shape: (7732, 60)


## 2.4 Save the data in the database

The dataframe contains features related to different themes:

* stellar properties
* transit parameters
* magnitudes at different spectral band
* location and motion
* pipeline statistics

We don't plan to use all these features but we will keep them and store them in different database tables to facilitate our future select queries during the EDA. Features related to the hosting stars will be stored in tables with the KIC number (`kepid` variable) as their primary unique key. Features related to the TCEs will be stored in tables with the KOI name (`kepoi_name` variable) as their primary key.

We are not building a datawarehouse so we autorize ourself to build a denormalized schema and skip some database best practices...

### 2.4.1 Save the stellar properties in a database table

In [25]:
# 1. CREATE A DATAFRAME WITH ONLY THE STELLAR PROPERTIES VARIABLES
# ----------------------------------------------------------------

# Stellar properties variables coming from the primary header
cols = [
    'kepid',
    'teff',
    'logg',
    'feh',
    'radius',
    'ebminusv',
    'av',
    'numtces',
    'quarters'
]

# Create a dataframe with only the stellar properties variables 
df = df_fits.reindex(columns=cols)

# Drop the duplicate records (because there are stars hosting multiple TCEs)
df.drop_duplicates(subset=['kepid'], keep='first', inplace=True)



# 2. CREATE THE DATABASE TABLE
# ----------------------------

# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS stellar_properties;')

# Create the stellar_properties table
query = '''
CREATE TABLE stellar_properties
(
    kepid TEXT PRIMARY KEY,
    teff REAL,
    logg REAL,
    feh REAL,
    radius REAL,
    ebminusv REAL,
    av REAL,
    numtces  INTEGER,
    quarters TEXT
);
'''
cursor.execute(query)


# 3. LOAD THE DATA IN THE TABLE
# -----------------------------

# Insert the source data in the fits_stellar_properties database table
df.to_sql(name='stellar_properties', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    stellar_properties;
'''
df = pd.read_sql_query(query, db)
print('Stellar Properties Database Table Shape: ', df.shape)
df.head()

Stellar Properties Database Table Shape:  (6606, 9)


,kepid,teff,logg,feh,radius,ebminusv,av,numtces,quarters
0,010797460,5850.0,4.426,0.14,1.040,0.142,0.441,2,11111111111111111000000000000000
1,010811496,5853.0,4.544,-0.18,0.868,0.120,0.373,1,11111101110111011000000000000000
2,010848459,5795.0,4.546,-0.52,0.803,0.122,0.378,1,11111110111011101000000000000000
3,010854555,6031.0,4.438,0.07,1.046,0.142,0.440,1,01111111111111111000000000000000
4,010872983,6046.0,4.486,-0.08,0.972,0.168,0.522,3,01111101110111011000000000000000


We can see that we have 7'732 TCEs around 6'606 stars.

### 2.4.2 Save the magnitude features in a database table

In [26]:
# 1. CREATE A DATAFRAME WITH ONLY THE MAGNITUDE VARIABLES
# -------------------------------------------------------

# Magnitude variables coming from the primary header
cols = [
    'kepid',
    'kepmag',
    'd51mag',
    'gmag',
    'hmag',
    'imag',
    'jmag',
    'kmag',
    'rmag',
    'zmag',
    'grcolor',
    'jkcolor',
    'gkcolor', 
]

# Create a dataframe with only the magnitude variables 
df = df_fits.reindex(columns=cols)

# Drop the duplicate records (because there are stars hosting multiple TCEs)
df.drop_duplicates(subset=['kepid'], keep='first', inplace=True)



# 2. CREATE THE DATABASE TABLE
# ----------------------------

# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS magnitudes;')

# Create the magnitudes table
query = '''
CREATE TABLE magnitudes
(
    kepid TEXT PRIMARY KEY,
    kepmag REAL,
    d51mag REAL,
    gmag REAL,
    hmag REAL,
    imag REAL,
    jmag REAL,
    kmag REAL,
    rmag REAL,
    zmag REAL,
    grcolor REAL,
    jkcolor REAL,
    gkcolor REAL  
);
'''
cursor.execute(query)


# 3. LOAD THE DATA IN THE TABLE
# -----------------------------

# Insert the source data in the fits_stellar_properties database table
df.to_sql(name='magnitudes', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    magnitudes;
'''
df = pd.read_sql_query(query, db)
print('Magnitudes Database Table Shape: ', df.shape)
df.head()

Magnitudes Database Table Shape:  (6606, 13)


,kepid,kepmag,d51mag,gmag,hmag,imag,jmag,kmag,rmag,zmag,grcolor,jkcolor,gkcolor
0,010797460,15.347,15.636,15.890,13.750,15.114,14.082,13.647,15.270,15.006,0.620,0.435,2.243
1,010811496,15.436,15.733,15.943,13.900,15.220,14.253,13.826,15.390,15.166,0.553,0.427,2.117
2,010848459,15.597,15.895,16.100,13.910,15.382,14.326,13.808,15.554,15.266,0.546,0.518,2.292
3,010854555,15.509,15.799,16.015,14.064,15.292,14.365,13.951,15.468,15.241,0.547,0.414,2.064
4,010872983,15.714,16.000,16.234,14.112,15.492,14.528,14.131,15.677,15.441,0.557,0.397,2.103


### 2.4.3 Save the location and motion features in a database table

In [27]:
# 1. CREATE A DATAFRAME WITH ONLY THE LOCATION AND MOTION VARIABLES
# -----------------------------------------------------------------

# Location and motion variables coming from the primary header
cols = [
    'kepid',
    'skygroup',
    'ra_obj',
    'dec_obj',
    'glon',
    'glat',
    'pmra',
    'pmdec',
    'pmtotal'
]

# Create a dataframe with only the location and motion variables 
df = df_fits.reindex(columns=cols)

# Drop the duplicate records (because there are stars hosting multiple TCEs)
df.drop_duplicates(subset=['kepid'], keep='first', inplace=True)



# 2. CREATE THE DATABASE TABLE
# ----------------------------

# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS location_motion;')

# Create the location_motion table
query = '''
CREATE TABLE location_motion
(
    kepid TEXT PRIMARY KEY,
    skygroup INTEGER, 
    ra_obj REAL,
    dec_obj REAL,
    glon REAL,
    glat REAL,
    pmra REAL,
    pmdec REAL,
    pmtotal REAL
);
'''
cursor.execute(query)


# 3. LOAD THE DATA IN THE TABLE
# -----------------------------

# Insert the source data in the fits_stellar_properties database table
df.to_sql(name='location_motion', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    location_motion;
'''
df = pd.read_sql_query(query, db)
print('Location and Motion Database Table Shape: ', df.shape)
df.head()

Location and Motion Database Table Shape:  (6606, 9)


,kepid,skygroup,ra_obj,dec_obj,glon,glat,pmra,pmdec,pmtotal
0,010797460,18,291.934230,48.14165,80.102268,14.237782,0.000,0.000,0.0000
1,010811496,34,297.004820,48.13413,81.651714,11.210840,0.000,0.000,0.0000
2,010848459,6,285.534611,48.28521,78.520972,18.224196,0.000,0.000,0.0000
3,010854555,24,288.754890,48.22620,79.293537,16.210389,0.000,0.000,0.0000
4,010872983,34,296.286140,48.22467,81.502660,11.676136,0.002,-0.006,0.0063


The following primary header features can be ommited without risk because they are catalog IDs or description
* keplerid
* object
* tmindex
* creator

### 2.4.4 Save the transit parameters in a database table

In [28]:
# 1. CREATE A DATAFRAME WITH ONLY THE TRANSIT PARAMETER VARIABLES
# ----------------------------------------------------------------

# Transit parameter variables coming from the TCE extension headers
cols = [
    'kepoi_name',
    'tperiod',
    'tdepth',
    'tdur',
    'indur',
    'impact',
    'inclin',
    'drratio',
    'radratio',
    'pradius',
]

# Create a dataframe with only the transit parameter variables 
df = df_fits.reindex(columns=cols)


# 2. CREATE THE DATABASE TABLE
# ----------------------------

# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS transit_parameters;')

# Create the transit_parameters table
query = '''
CREATE TABLE transit_parameters
(
    kepoi_name TEXT PRIMARY KEY,
    tperiod REAL,
    tdepth REAL,
    tdur REAL,
    indur REAL,
    impact REAL,
    inclin REAL,
    drratio REAL,
    radratio REAL,
    pradius REAL
);
'''
cursor.execute(query)


# 3. LOAD THE DATA IN THE TABLE
# -----------------------------

# Insert the source data in the fits_stellar_properties database table
df.to_sql(name='transit_parameters', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    transit_parameters;
'''
df = pd.read_sql_query(query, db)
print('Transit Parameters Database Table Shape: ', df.shape)
df.head()

Transit Parameters Database Table Shape:  (7732, 10)


,kepoi_name,tperiod,tdepth,tdur,indur,impact,inclin,drratio,radratio,pradius
0,K00752.01,9.488028,630.381630,3.418783,0.659365,0.934177,84.169681,9.196221,0.028956,2.929099
1,K00752.02,54.418374,893.123959,4.585753,0.257789,0.711630,89.395301,67.428761,0.029346,2.968514
2,K00753.01,19.899143,10630.409762,1.776158,0.888079,0.929919,89.029114,54.880784,0.129501,12.266182
3,K00754.01,1.736955,7874.849223,2.341290,1.170645,0.933470,74.673384,3.531577,0.108508,9.365962
4,K00755.01,2.525622,578.297482,1.624692,0.070214,0.690533,85.604302,9.009601,0.023557,2.688826


### 2.4.5 Save the TCE pipeline statistics in a database table

In [29]:
# 1. CREATE A DATAFRAME WITH ONLY THE PIPELINE STATISTICS VARIABLES
# -----------------------------------------------------------------

# Pipeline statistics variables coming from the TCE extension headers
cols = [
    'kepoi_name',
    'tsnr',
    'maxmes',
    'maxses',
    'meddetr',
    'cdpp3_0',
    'cdpp6_0',
    'cdpp12_0',
    'ntrans',
    'convrge'
]

# Create a dataframe with only the pipeline statistics variables 
df = df_fits.reindex(columns=cols)


# 2. CREATE THE DATABASE TABLE
# ----------------------------

# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS pipeline_statistics;')

# Create the pipeline_statistics table
query = '''
CREATE TABLE pipeline_statistics
(
    kepoi_name TEXT PRIMARY KEY,
    tsnr REAL,
    maxmes REAL,
    maxses REAL,
    meddetr REAL,
    cdpp3_0 REAL,
    cdpp6_0 REAL,
    cdpp12_0 REAL,
    ntrans INTEGER,
    convrge BOOLEAN
);
'''
cursor.execute(query)


# 3. LOAD THE DATA IN THE TABLE
# -----------------------------

# Insert the source data in the fits_stellar_properties database table
df.to_sql(name='pipeline_statistics', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    pipeline_statistics;
'''
df = pd.read_sql_query(query, db)
print('Pipeline Statistics Database Table Shape: ', df.shape)
df.head()

Pipeline Statistics Database Table Shape:  (7732, 10)


,kepoi_name,tsnr,maxmes,maxses,meddetr,cdpp3_0,cdpp6_0,cdpp12_0,ntrans,convrge
0,K00752.01,31.178432,28.470819,5.135849,17.639999,224.206421,184.823334,163.164536,142,1
1,K00752.02,20.690699,20.109507,7.027669,23.520000,224.206421,184.823334,163.164536,25,1
2,K00753.01,181.560364,187.449097,37.159767,9.800000,285.547546,257.448669,271.064423,56,1
3,K00754.01,530.372681,541.895081,39.066551,11.760000,242.146011,183.067200,140.485062,621,1
4,K00755.01,37.310295,33.191898,4.749945,8.820000,272.847504,222.750381,192.141464,515,1


The following TCE extension features can be ommited without risk because they are experimental description features

* tepoch: transit epoch in bkjd
* tstart: observation start time in BJD-BJDREF
* tstop: observation stop time in BJD-BJDREF
* telapse: TSTOP - TSTART
* livetime: TELAPSE multiplied by DEADC
* exposure: time on source

# 3. TCE status database table 

During the EDA, we will drop observations. Keeping track of them and the reason they have been dropped is a good practice and easy way to explain and/or justify models results, expected or not. So, we will add a status technical table hosting one record per TCE, indicating if it has been dropped during the EDA and for which reason.

In [30]:
# Drop the table if it already exists (easy to iterate in development)
cursor.execute('DROP TABLE IF EXISTS tce_status;')

query = '''
CREATE TABLE tce_status
(
    kepoi_name TEXT PRIMARY KEY,
    kepid TEXT NOT NULL,
    excluded BOOLEAN DEFAULT 0,
    exclusion_reason TEXT DEFAULT "",
    FOREIGN KEY (kepid) REFERENCES stellar_properties (kepid)
        ON DELETE CASCADE ON UPDATE NO ACTION,
    FOREIGN KEY (kepid) REFERENCES magnitudes (kepid)
        ON DELETE CASCADE ON UPDATE NO ACTION,
    FOREIGN KEY (kepid) REFERENCES location_motion (kepid)
        ON DELETE CASCADE ON UPDATE NO ACTION        
);
'''
cursor.execute(query)

In [32]:
# Load the table with the list of KOIs
kois.loc[:, ['kepoi_name', 'kepid']].to_sql(name='tce_status', con=db, if_exists='append', index=False)

# Check the loading
query = '''
SELECT *
FROM
    tce_status;
'''
df = pd.read_sql_query(query, db)
print('TCE Status Database Table Shape: ', df.shape)
df.head()

TCE Status Database Table Shape:  (7732, 4)


,kepoi_name,kepid,excluded,exclusion_reason
0,K00752.01,010797460,0,
1,K00752.02,010797460,0,
2,K00753.01,010811496,0,
3,K00754.01,010848459,0,
4,K00755.01,010854555,0,


In [33]:
# Commit and close the database
db.commit()
db.close()